In [ ]:
#Citation for the below code
'''
Title: CCMEO Microplot Clipper Script
Authors: Galen Richardson, Julie Lovitt, Sylvain Leblanc
Date: 02-25-2021
Version: 1.8
https://www.nrcan.gc.ca/science-and-data/research-centres-and-labs/canada-centre-remote-sensing/21749
'''


In [140]:
import numpy as np
from PIL import Image, ImageFilter
import os,cv2,time,shutil,csv
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 150 
%matplotlib inline

from sklearn.linear_model import TheilSenRegressor,RANSACRegressor
from sklearn import linear_model

In [141]:
#Directory where the photos ready to be clipped are located
photo_dir=r'C:\Users\gamr0\Desktop\clip'

#Directory where the clipped photos and b/w photos(from stage 1) are output
clip_dir = r'C:\Users\gamr0\Desktop\m'

#Variable which defines what kind of box (red/orange, or blue). valid enteries are "red" or "blue"
red_blue = "red"

## Stage 1: Defining the Box

Directory Setup

In [142]:
def Testing_dir_setup(photo_dir, clip_dir):
    global in_list
    global bw_list
    global clip_list
    #if bw/clip folders exist delete the contents, if not create new folders
    if os.path.exists(clip_dir):
        shutil.rmtree(clip_dir)
        os.makedirs(clip_dir)
    else:
        os.makedirs(clip_dir)
    in_list=[]
    bw_list=[]
    clip_list=[]
    for path,subdir,files in os.walk(photo_dir):
        for file in files:
            in_list.append(photo_dir+'\\'+file)
            bw_list.append(clip_dir+'\\'+file)
            clip_list.append(clip_dir+'\\'+file)

Resizing and bluring

In [143]:
def open_resize(input):
    img=Image.open(input)
    IMG_W,IMG_H = 3000,3000
    #resizing to known aspect ratios and approx image sizes
    if (img.size[0]/ img.size[1]) >1.19:
        IMG_W,IMG_H= 3600,3000
    if (img.size[0]/ img.size[1]) >1.29:
        IMG_W,IMG_H = 3840,3000
    if (img.size[0]/ img.size[1]) >1.7:
        IMG_W,IMG_H = 3840,2160
    if (img.size[1]/ img.size[0]) >1.19:
        IMG_W,IMG_H = 3000,3600
    if (img.size[1]/ img.size[0]) >1.29:
        IMG_W,IMG_H = 3000,3840
    if (img.size[1]/ img.size[0]) >1.7:
        IMG_W,IMG_H = 2160,3840
    img=img.resize((IMG_W,IMG_H), Image.ANTIALIAS)
    imgcv = np.array(img)
    #adding blur
    blur = cv2.GaussianBlur(imgcv,(5,5),0)
    blur = cv2.GaussianBlur(blur,(11,11),0)#(old 10)
    r, g, b = cv2.split(blur)
    return r,g,b

Moving window which looks for smoothness. Erosion, Dilation, mode filtering to remove outliers in smooth image. Smooth image is then thresholded compares to r,g,b targets.

In [144]:
def moving_window_range(input_bw,g,b):
    #defining data and window size of
    data,win_size,sub_win=input_bw,(12,12),9
    rows = data.shape[0] - win_size[0] + 1
    cols = data.shape[1] - win_size[1] + 1
    slices = []
    #creating slics (mini windows to work on)
    for i in range(win_size[0]):
        for j in range(win_size[1]):
            slices.append(data[i:rows+i, j:cols+j])
    #stacking the slices together, creating out image
    stacked = np.dstack(slices)
    outdata = np.zeros(((data.shape[0]-sub_win),(data.shape[1]-sub_win)), np.float32)
    #np.ptp is finding the range of each slice, setting it to pix
    outdata[1:-1, 1:-1] = np.ptp(stacked, 2)
    out=outdata.astype(np.uint8)
    outn= (255-out)
    outn[outn <249]=0
    outn[outn >=249]=255
    #eroding, dilating, and moding to reduce noise
    kernel = np.ones((3,3),np.uint8)
    e1 = cv2.erode(outn,kernel,iterations = 2,borderType = 0)
    d1 = cv2.dilate(e1,kernel,iterations = 2,borderType = 0)
    pil=(Image.fromarray(d1)).filter(ImageFilter.ModeFilter(size=15))#old 15
    cv=np.array(pil)
    #finding coord of shrunken image from moving windo
    x=input_bw.shape[0]-sub_win
    y=input_bw.shape[1]-sub_win
    #resizing r,g,b bands to smaller image
    rnew=input_bw.copy()
    b=cv2.resize(b,(y,x))
    g=cv2.resize(g,(y,x))
    rnew=cv2.resize(rnew,(y,x))
    #thresholding bands, note that red threshold is inverted because we want red
    rnew[rnew <210]=100 
    rnew[rnew >=210]=0
    rnew[rnew ==100]=255
    b[b <230]=0 
    b[b >=230]=255 
    g[g <250]=0
    g[g >=250]=255 
    #subtracting thres bands from smoothed image to isolate smooth reddish pix
    combo=cv-b-g-rnew

    return combo

Script which is used to set up the process

In [145]:
def bw_processing():
    start1 = time.time()
    Testing_dir_setup(photo_dir,clip_dir)
    for i in range (len(in_list)):
        print('working on photo '+str(i+1))
        r,g,b=open_resize(in_list[i])
        if red_blue=='red':
            out_bw = moving_window_range(r,g,b)
        if red_blue=='blue':
            out_bw = moving_window_range(b,g,r)
        cv2.imwrite(bw_list[i],out_bw)
    end1 = time.time()
    print('Done in '+str("%.2f"%(end1 - start1))+' sec')

In [146]:
bw_processing()

working on photo 1
working on photo 2
working on photo 3
working on photo 4
working on photo 5
Done in 127.84 sec


## Stage 2: Clip from Linear Regression

Defining the center, interior box size around center pixel

In [147]:
def crop_basics(bw_filepath,og_filepath):
    #parameters for rest of script
    global imgo
    global imgbw
    global centbw
    global boxsize
    #size of box around the center pixel,reduce if not getting good results
    boxsize=500
    imgbw=Image.open(bw_filepath)
    imgo=Image.open(og_filepath)
    imgo=imgo.resize((imgbw.size[0],imgbw.size[1]))
    centbw=(int(imgbw.size[0]/2),int(imgbw.size[1]/2))

Creation of lists of coordinates from center box to outer box

In [148]:
def ret_lx_pos(start_posx, start_posy):
    #returns leftx position
    left_x=[]
    for i in range(start_posx,5,-1):
        px = imgbw.getpixel((i,start_posy))
        left_x.append(px)
    position=0
    for i in left_x:
        if i >200:
            return (start_posx-position,start_posy)
        position=position+1
def L_cord_list():
    #creates a list of all the leftx positions
    L_cord=[]
    for i in range(centbw[1]-boxsize,centbw[1]+boxsize,3):
        x=ret_lx_pos(centbw[0], i)
        if x != None:
            L_cord.append(x)
    return L_cord

In [149]:
def ret_rx_pos(start_posx, start_posy):
    #returns rightx position
    right_x=[]
    for i in range (start_posx,(imgbw.size[0]-5),1):
        px = imgbw.getpixel((i,start_posy))
        right_x.append(px)
    position=0
    for i in right_x:
        if i >200:
            return (start_posx+position,start_posy)
        position=position+1
def R_cord_list():
    #creates a list of all the rightx positions
    R_cord=[]
    for i in range(centbw[1]-boxsize,centbw[1]+boxsize,3):
        x=ret_rx_pos(centbw[0], i)
        if x != None:
            R_cord.append(x)
    return R_cord

In [150]:
def ret_dy_pos(start_posx, start_posy):
    #returns down y position 
    down_y=[]
    for i in range(start_posy,(imgbw.size[1]-5),1):
        px = imgbw.getpixel((start_posx,i))
        down_y.append(px)
    position=0
    for i in down_y:
        if i >200:
            return (start_posx,start_posy+position)
        position=position+1
def d_cord_list():
    #creates a list of all down y position
    d_cord=[]
    for i in range(centbw[0]-boxsize,centbw[0]+boxsize,3):
        x=ret_dy_pos(i, centbw[1])
        if x != None:
            d_cord.append(x)
    return d_cord

In [151]:
def ret_uy_pos(start_posx, start_posy):
    #returns up y postition
    up_y=[]
    for i in range(start_posy,5,-1):
        px = imgbw.getpixel((start_posx,i))
        up_y.append(px)
    position=0
    for i in up_y:
        if i >200:
            return (start_posx,start_posy-position)
        position=position+1
def u_cord_list():
    #creates a list of all up y  positions
    u_cord=[]
    for i in range(centbw[0]-boxsize,centbw[0]+boxsize,3):
        x=ret_uy_pos(i, centbw[1])
        if x != None:
            u_cord.append(x)
    return u_cord

Defining the linear regressions used to define the box sides

In [152]:
def RL_theil(cord):
    #regression function used for left and right sides
    x,y = zip(*cord)
    x=np.array((x)).reshape((-1, 1))
    y=list(y)
    modelT = TheilSenRegressor(max_iter=600).fit(x, y)
    modelR= RANSACRegressor(max_trials=5000).fit(x,y)
    modelL = linear_model.LinearRegression().fit(x, y)
    tscore,rscore,lscore=abs(modelT.score(x, y)),abs(modelR.score(x, y)),abs(modelL.score(x, y))
    #if the linear regression is good (over .95), it usually is the best score
    if lscore > .95:
        print('Linear init')
        return modelL.coef_,modelL.intercept_
    #if not linear, usually thiel over .9 is 2nd best
    if tscore > .9:
        print('Theil init')
        return modelT.coef_,modelT.intercept_
    #let the scores decide which one is best
    if tscore > rscore and lscore:
        print('Theil')
        return modelT.coef_,modelT.intercept_
    if rscore > tscore and lscore:
        print('RANSAC')
        return modelR.estimator_.coef_,modelR.estimator_.intercept_
    if lscore > tscore and rscore:
        print('Linear')
        return modelL.coef_,modelL.intercept_

In [153]:
def UD_theil(cord):
    #regression function used for up and down sides
    #inverted y and x to avoid infinate slope
    y,x = zip(*cord)
    x=np.array((x)).reshape((-1, 1))
    y=list(y)
    modelT = TheilSenRegressor(max_iter=600).fit(x, y)
    modelR= RANSACRegressor(max_trials=5000).fit(x,y)
    modelL = linear_model.LinearRegression().fit(x, y)
    tscore,rscore,lscore=abs(modelT.score(x, y)),abs(modelR.score(x, y)),abs(modelL.score(x, y))
    #if the linear regression is good (over .95), it usually is the best score
    if lscore > .95:
        print('Linear init')
        return modelL.coef_,modelL.intercept_
    #if not linear, usually thiel over .9 is 2nd best
    if tscore > .9:
        print('Theil init')
        return modelT.coef_,modelT.intercept_
    #let the scores decide which one is best
    if tscore > rscore and lscore:
        print('Theil')
        return modelT.coef_,modelT.intercept_
    if rscore > tscore and lscore:
        print('RANSAC')
        return modelR.estimator_.coef_,modelR.estimator_.intercept_
    if lscore > tscore and rscore:
        print('Linear')
        return modelL.coef_,modelL.intercept_

Equating and up/down regression to a left/right regression to find the corners

In [154]:
def Poly_corns(LR_slope, LR_int, UD_Slope, UD_int):
    #using the slopes and intersepts, this functions finds polygon corners
    #where an up/down reg meets a left/right
    y=((LR_slope*UD_int)+LR_int)/(1-(LR_slope*UD_Slope))
    x=UD_Slope*y+UD_int
    return abs(int(x)),abs(int(y))

Function to get the 4 corners 

In [155]:
def get_corners():
    #this function finds the coordinates of each side
    L_cord=L_cord_list()
    R_cord=R_cord_list()
    D_cord=d_cord_list()
    U_cord=u_cord_list()
    #Contingency planning if there are no coordinates
    if L_cord == []:
        return 'no'
    if R_cord == []:
        return 'no'
    if D_cord == []:
        return 'no'
    if U_cord == []:
        return 'no'
    #finds the regression for each side
    slopeR,interceptR = RL_theil(R_cord)
    slopeL,interceptL = RL_theil(L_cord)
    slopeD,interceptD = UD_theil(D_cord)
    slopeU,interceptU = UD_theil(U_cord)
    #finds the corners where each regression meets
    UL=list(Poly_corns(slopeL, interceptL, slopeU, interceptU))
    UR=list(Poly_corns(slopeR, interceptR, slopeU, interceptU))
    DL=list(Poly_corns(slopeL, interceptL, slopeD, interceptD))
    DR=list(Poly_corns(slopeR, interceptR, slopeD, interceptD))
    return UL,DL,DR,UR

Given the corners, this function creates a mask, clips to mask, then does a final bbox crop

In [156]:
def process_clips(Corners):
    global imgo
    global imgbw
    print (Corners)
    image=imgo.resize((imgbw.size[0],imgbw.size[1]))
    my_img = np.zeros((imgbw.size[1], imgbw.size[0], 3), dtype = "uint8")
    pts = np.array(Corners, np.int32)
    pts = pts.reshape((-1,1,2))
    #creation of mask from corner points (white)
    cv2.fillPoly(my_img, [pts], (255,255,255))
    kernel = np.ones((10,10),np.uint8)
    #shrinking the mask a bit so minimal box is in frma
    e1 = cv2.erode(my_img,kernel,iterations = 16,borderType = 0)
    #resizing and applying the mask
    image=imgo.resize((imgbw.size[0],imgbw.size[1]))
    result = cv2.bitwise_and(np.array(image), e1)
    im = Image.fromarray(result)
    y_nonzero, x_nonzero, _ = np.nonzero(im)
    final = im.crop((np.min(x_nonzero), np.min(y_nonzero), np.max(x_nonzero), np.max(y_nonzero)))
    return final

The standard deviation of the angles is how we can determine how skewed the angles of the box are. High angle sdv means that either the box in the image is skewed or there was a bad clip. This function determines sdev

In [157]:
def angle_sdv(output_corners):
    ang_list=[]
    i,j,k=0,0,0
    for i in range (4):
        #determining the angles of the corners
        j,k=1,2
        if i==2:
            k= -2
        if i==3:
            j,k=-3,-2
        a,b,c = np.array(output_corners[i]),np.array(output_corners[i+j]),np.array(output_corners[i+k])
        ba = a - b
        bc = c - b
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(cosine_angle)
        ang_list.append(np.degrees(angle))
    #returning the standard deviation of the angle list
    return np.std(ang_list)

csv logging scripts

In [158]:
def cv_log(file, score):
    with open(clip_dir+'\potential_misclips.csv','a',newline='')as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow([file,score])

Defining the workflow

In [159]:
def Clipping_Workflow():
    #the workflow of the clipping, uses in_list and BW_list as sources
    global in_list
    global bw_list
    global clip_list
    start2 = time.time()
    scores=[]
    with open(clip_dir+'\potential_misclips.csv','w',newline='')as csvfile:
        writer = csv.writer(csvfile, delimiter=',',
            quotechar='|', quoting=csv.QUOTE_MINIMAL)
        writer.writerow(["Filename","SDEV Angle Score","Higher SDEV means greater variance in corner angles. This can because of bad clip or skewed image"])
    for i in range (len(in_list)):
        print('working on photo '+str(i+1))
        print(in_list[i])
        crop_basics(bw_list[i],in_list[i])
        output_corners=get_corners()
        if output_corners == "no":
            print('Skipping'+str(in_list[i]))
            cv_log(in_list[i],'THIS IMAGE WAS NOT CROPPED DUE TO MISSING SIDE')
            ogimg=Image.open(in_list[i])
            ogimg.save(clip_list[i]) 
            continue
        score=angle_sdv(output_corners)
        if score >5:
            print((str(int(score)))+' angle SDEV')
        scores.append(score)
        if score >5:
            if score>15:
                cv_log(clip_list[i],"SKIPPED DUE TO HIGH SDEV")
                ogimg=Image.open(in_list[i])
                ogimg.save(clip_list[i]) 
                continue
            cv_log(clip_list[i],score)
            final_img=process_clips(output_corners)
            final_img.save(clip_list[i])
        else:
            final_img=process_clips(output_corners)
            final_img.save(clip_list[i]) 
        #display
        #plt.figure(figsize = (5,5))
        #imgplot = plt.imshow(final_img)
        #plt.show()
    end2 = time.time()
    print ("Done in "+str("%.2f"%(end2 - start2))+' sec')
    print("Average Angle Standard Deviation = "+str(int((sum(scores) / len(scores)))))

In [160]:
Clipping_Workflow()

working on photo 1
C:\Users\gamr0\Desktop\clip\H1C.JPG
RANSAC
Linear init
Linear init
Linear init
([430, 232], [534, 2940], [3111, 2780], [3018, 118])
working on photo 2
C:\Users\gamr0\Desktop\clip\H1NW.JPG
SkippingC:\Users\gamr0\Desktop\clip\H1NW.JPG
working on photo 3
C:\Users\gamr0\Desktop\clip\H1SE.JPG
RANSAC
Theil
Linear init
Linear init
([482, 149], [575, 2738], [3083, 2689], [3044, 11])
working on photo 4
C:\Users\gamr0\Desktop\clip\M4C.JPG
RANSAC
Linear init
RANSAC
Theil
([232, 114], [450, 2808], [2901, 2741], [2938, 46])
working on photo 5
C:\Users\gamr0\Desktop\clip\S9P2.JPG
Theil init
RANSAC
Theil init
Theil init
16 angle SDEV
Done in 20.00 sec
Average Angle Standard Deviation = 5
